# 3.0 Modelado y Predicción de Demanda

En este notebook, entrenaremos varios de Machine Learning para predecir la demanda de taxis. Utilizaremos las características creadas en el notebook anterior y evaluaremos el rendimiento de los modelos con la métrica RMSE.

In [1]:
from pathlib import Path
import sys

current_working_directory = Path.cwd()
project_root = current_working_directory.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
# Bibliotecas necesarias
import pandas as pd
import os

from src.models.train_model import train_best_model
from src.utils.helpers import print_rmse

### Carga de Datos con Características

Cargamos el DataFrame que ya procesado.

In [5]:
features_data_path = '../data/processed/features_taxi_data.parquet'

if not os.path.exists(features_data_path):
    print(f"Error: No se encontró el archivo de características en {features_data_path}.")
    print("Por favor, ejecuta el notebook '2.0-ingenieria-caracteristicas-preparacion.ipynb' primero.")
    df_features = None
else:
    df_features = pd.read_parquet(features_data_path)
    print(f"Datos cargados desde: {features_data_path}")
    print(df_features.head())

Datos cargados desde: ../data/processed/features_taxi_data.parquet
                     num_orders  year  month  day  dayofweek  hour  lag_1  \
datetime                                                                    
2018-03-01 00:00:00         124  2018      3    1          3     0    NaN   
2018-03-01 01:00:00          85  2018      3    1          3     1  124.0   
2018-03-01 02:00:00          71  2018      3    1          3     2   85.0   
2018-03-01 03:00:00          66  2018      3    1          3     3   71.0   
2018-03-01 04:00:00          43  2018      3    1          3     4   66.0   

                     lag_2  lag_3  lag_4  ...  lag_16  lag_17  lag_18  lag_19  \
datetime                                  ...                                   
2018-03-01 00:00:00    NaN    NaN    NaN  ...     NaN     NaN     NaN     NaN   
2018-03-01 01:00:00    NaN    NaN    NaN  ...     NaN     NaN     NaN     NaN   
2018-03-01 02:00:00  124.0    NaN    NaN  ...     NaN     NaN     NaN

### Entrenamiento del Modelo

Entrenaremos el modelo utilizando la función `train_best_model` que encapsula la división de datos, la búsqueda de hiperparámetros, y el entrenamiento para un modelo entrando con LightGBM. El modelo entrenado será guardado.

In [6]:
if df_features is not None:
    TARGET_COLUMN = 'num_orders'
    MODEL_SAVE_PATH = '../models/best_taxi_demand_model.pkl'
    TEST_SIZE = 0.1
    STATE = 31415

    # Entrenamiento del modelo
    best_model, features_test, target_test, predictions = train_best_model(
        df_features.copy(),
        TARGET_COLUMN,
        test_size=TEST_SIZE,
        state=STATE,
        model_save_path=MODEL_SAVE_PATH
    )

    print('Evaluación del modelo final en el conjunto de pruebas:')
    rmse_test = print_rmse(target_test, predictions['test'])

    print(f"El RMSE en el conjunto de prueba es: {round(rmse_test, 2)}")
    if rmse_test <= 48:
        print("¡El modelo cumple el requisito de RMSE (<= 48)!")
    else:
        print("El modelo NO cumple el requisito de RMSE (debe ser <= 48). Se necesita optimización adicional.")

    predictions_test_df = pd.DataFrame({
        'real': target_test,
        'predicted': predictions['test']
    })
    predictions_test_df.to_parquet('../data/processed/test_predictions.parquet', index=True)
    print(f"Predicciones del conjunto de prueba guardadas en: data/processed/test_predictions.parquet")

else:
    print("No se pudo entrenar el modelo porque los datos de características no se cargaron.")


Tamaños conjunto de entrenamiento: (3952, 30)
Tamaños conjunto de prueba: (440, 30)

Buscando los mejores hiperparámetros para LightGBM...
Mejores hiperparámetros encontrados:
{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'num_leaves': 10}

Entrenando el modelo final con los mejores hiperparámetros...
Tiempo de entrenamiento: 0.0366 s
Tiempo de predicción: 0.0023 s
Modelo entrenado guardado en: ../models/best_taxi_demand_model.pkl
Evaluación del modelo final en el conjunto de pruebas:
El RMSE calculado es: 41.21
El RMSE en el conjunto de prueba es: 41.21
¡El modelo cumple el requisito de RMSE (<= 48)!
Predicciones del conjunto de prueba guardadas en: data/processed/test_predictions.parquet
